In [ ]:
import sys 
sys.path.append("../..")
sys.path.append("..")

from importlib import reload

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)


model = HookedTransformer.from_pretrained(
    "gpt2-small",
    # "tiny-stories-2L-33M",
    # "attn-only-2l",
    # center_unembed=True,
    # center_writing_weights=True,
    # fold_ln=True,
    # refactor_factored_attn_matrices=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)


# path = "./artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/1100001280_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152.pt"
path = "/Users/josephbloom/GithubRepositories/mats_sae_training/artifacts/sparse_autoencoder_gelu-2l_blocks.0.hook_mlp_out_32768:v40/final_sparse_autoencoder_gelu-2l_blocks.0.hook_mlp_out_32768.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

print(sparse_autoencoder.cfg)


# sanity check
text = "Many important transition points in the history of science have been moments when science 'zoomed in.' At these points, we develop a visualization or tool that allows us to see the world in a new level of detail, and a new field of science develops to study the world through this lens."
model(text, return_type="loss")


In [ ]:
from sae_training.utils import LMSparseAutoencoderSessionloader
model, sparse_autoencoder, activation_store = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path
)

In [10]:
from importlib import reload
from sae_training.geom_median.src import geom_median
reload(geom_median)


with torch.no_grad():
    
    all_activations = activation_store.storage_buffer.detach()

    out = geom_median.torch.compute_geometric_median(
        activation_store.storage_buffer.detach().cpu(), skip_typechecks=True, maxiter=100, per_component=False,
        ).median
    
    # get the distance between the median and each activation
    dataset_mean = all_activations.mean(dim=0).detach().cpu()
    dataset_mean_distances = all_activations.median(0).values.detach().cpu() - dataset_mean
    dataset_mean_distances_mean = dataset_mean_distances.mean().item()
    
    
    med_distances = all_activations.median(0).values.detach().cpu() -  out.to("mps").detach().cpu()
    med_distances_mean = med_distances.mean().item()
    
    dec_distance = all_activations.median(0).values.detach().cpu() -  sparse_autoencoder.b_dec.detach().cpu()
    dec_distance_mean = dec_distance.mean().item()
    
    
    
    
    print(med_distances_mean, dec_distance_mean, dataset_mean_distances_mean)


    px.line(dataset_mean_distances, title = "dataset mean distances").show()
    px.line(med_distances, title = "median distances").show()
    px.line(dec_distance, title = "decoder distances").show()
                

Objective value: 11.3754:  14%|█▍        | 14/100 [00:10<01:04,  1.34it/s]


0.000584263529162854 0.03458179906010628 0.0005843249382451177


In [ ]:
abs_activation_mean = all_activations.median(0).values.abs().detach().cpu()
outlier_dims = (abs_activation_mean > 0.8).nonzero().squeeze().tolist()
fig = px.line(abs_activation_mean, title = "median activations")
# add vertical lines for outlier dims
for dim in outlier_dims:
    fig.add_vline(dim, line_width=1, line_dash="dash", line_color="red")
    
fig.show()

In [ ]:
px.line(activations[0].detach().cpu())

In [ ]:
px.strip(all_activations[:, outlier_dims].detach().cpu(), 
         color="variable",
         orientation="h",
         title = "Outlier Dimension Activations").show()
for dim in outlier_dims:
    print((all_activations[:, dim].abs() > 2).float().mean().item())

In [ ]:
from umap import UMAP
import plotly.graph_objects as go
# umap = UMAP(n_components=2, n_neighbors=10, min_dist=0.1, metric="cosine")

# umap.fit(all_activations.detach().cpu().numpy())

# umap_embedding = umap.embedding_

# project the median and decoder onto the umap embedding
median_embedding = umap.transform(out.unsqueeze(0).to("cpu").detach().numpy())
decoder_embedding = umap.transform(sparse_autoencoder.b_dec.unsqueeze(0).to("cpu").detach().numpy())
mean_embedding = umap.transform(dataset_mean.unsqueeze(0).detach().cpu().numpy())


fig = go.Figure()


fig.add_scatter(
    x=umap_embedding[:, 0],
    y=umap_embedding[:, 1],
    mode="markers",
    name="activations",
    marker=dict(color="grey"),
    opacity=0.3,
)

fig.layout.showlegend = True
fig.add_scatter(
    x=median_embedding[:, 0],
    y=median_embedding[:, 1],
    mode="markers",
    name="median",
    marker=dict(color="red"),
    
)

fig.add_scatter(
    x=decoder_embedding[:, 0],
    y=decoder_embedding[:, 1],
    mode="markers",
    name="decoder",
    marker=dict(color="green"),
)

fig.add_scatter(
    x=mean_embedding[:, 0],
    y=mean_embedding[:, 1],
    mode="markers",
    name="mean",
    marker=dict(color="blue"),
)

fig.show()


